# L.A. hospital occupancy data over time

#### An analysis of HHS hospital data, by @datagraphics and @stiles.

#### **Questions?** [matt.stiles@latimes.com](matt.stiles@latimes.com) \\ 310.529.8749

---

### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Fetch timeseries data

In [3]:
df = pd.read_csv(
    "output/trimmed_timeseries.csv",
    parse_dates=["week"],
    infer_datetime_format=True,
    dtype={"fips": str},
)

In [4]:
df.head()

,hospital,hospital_subtype,ccn,week,county,fips,total_beds,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
0,Kaiser Foundation Hospital - Fremont,Short Term,050512,2021-01-29,Alameda,001,94.0,12.0,8.0,1.0,0.92,34.0,85.0,0.40,Bay Area
1,Kaiser Foundation Hospital - Oakland/Richmond,Short Term,050075,2021-01-29,Alameda,001,270.0,31.0,22.0,1.0,0.97,64.0,248.0,0.26,Bay Area
2,Washington Hospital,Short Term,050195,2021-01-29,Alameda,001,230.0,29.0,15.0,0.0,1.00,59.0,196.0,0.30,Bay Area
3,Kaiser Foundation Hospital - San Leandro,Short Term,050777,2021-01-29,Alameda,001,182.0,19.0,11.0,1.0,0.95,42.0,165.0,0.25,Bay Area
4,St Rose Hospital,Short Term,050002,2021-01-29,Alameda,001,95.0,14.0,6.0,2.0,0.86,24.0,58.0,0.41,Bay Area


In [5]:
len(df)

In [10]:
df.week.value_counts()

2020-12-11    349
2020-12-18    349
2021-01-01    348
2021-01-22    348
2020-09-25    348
2020-08-07    348
2020-08-28    348
2020-09-18    348
2021-01-15    348
2021-01-08    348
2020-12-25    347
2021-01-29    347
2020-09-04    347
2020-12-04    347
2020-08-14    347
2020-08-21    347
2020-11-06    346
2020-07-31    346
2020-09-11    346
2020-10-30    345
2020-10-16    345
2020-10-02    345
2020-11-20    345
2020-11-13    345
2020-11-27    345
2020-10-23    344
2020-10-09    344
Name: week, dtype: int64

---

### How have L.A. hositals changed over time? 

In [11]:
surge_la = df[df["week"] > "2020-11-13"]

In [12]:
alt.Chart(surge_la).mark_line().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(format="%", title="", tickCount=6),
    ),
    color=alt.Facet("hospital"),
).properties(
    width=1000, height=1600, title="Share of all patients with Covid in SoCal Counties"
)

alt.Chart(...)